In [1]:
import os
import numpy as np
import pandas as pd

from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from sklearn import ensemble
from sklearn import model_selection

from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression

In [2]:
# 弃掉不需要的列
def drop_col_not_req(df, cols):
    df.drop(cols, axis=1, inplace=True)


# 建立Fare Category
def fare_category(fare):
    if fare <= 4:
        return 'Very_Low_Fare'
    elif fare <= 10:
        return 'Low_Fare'
    elif fare <= 30:
        return 'Med_Fare'
    elif fare <= 45:
        return 'High_Fare'
    else:
        return 'Very_High_Fare'


# 建立PClass Fare Category
def pclass_fare_category(df, Pclass_1_mean_fare, Pclass_2_mean_fare, Pclass_3_mean_fare):
    if df['Pclass'] == 1:
        if df['Fare'] <= Pclass_1_mean_fare:
            return 'Pclass_1_Low_Fare'
        else:
            return 'Pclass_1_High_Fare'
    elif df['Pclass'] == 2:
        if df['Fare'] <= Pclass_2_mean_fare:
            return 'Pclass_2_Low_Fare'
        else:
            return 'Pclass_2_High_Fare'
    elif df['Pclass'] == 3:
        if df['Fare'] <= Pclass_3_mean_fare:
            return 'Pclass_3_Low_Fare'
        else:
            return 'Pclass_3_High_Fare'


# 建立Family Size Category
def family_size_category(family_size):
    if family_size <= 1:
        return 'Single'
    elif family_size <= 3:
        return 'Small_Family'
    else:
        return 'Large_Family'


# 建立Age_Group_category
def age_group_category(age):
    if age <= 1:
        return 'Baby'
    elif age <= 4:
        return 'Toddler'
    elif age <= 12:
        return 'Child'
    elif age <= 19:
        return 'Teenager'
    elif age <= 30:
        return 'Adult'
    elif age <= 50:
        return 'Middle_Aged'
    elif age < 60:
        return 'Senior_Citizen'
    else:
        return 'Old'


# 建立Name_Length_Category
def name_length_category(name_len):
    if name_len <= 19:
        return 'Very_Short_Name'
    elif name_len <= 28:
        return 'Short_Name'
    elif name_len <= 45:
        return 'Medium_Name'
    else:
        return 'Long_Name'


In [3]:
# 填充NaN值
# 使用GradientBoostingRegressor和LinearRegression来填充Age值
def fill_missing_age(missing_age_train, missing_age_test):
    missing_age_X_train = missing_age_train.drop(['Age'], axis=1)
    missing_age_Y_train = missing_age_train['Age']
    missing_age_X_test = missing_age_test.drop(['Age'], axis=1)

    gbm_reg = ensemble.GradientBoostingRegressor(random_state=42)
    gbm_reg_param_grid = {'n_estimators': [2000], 'max_depth': [3],'learning_rate': [0.01], 'max_features': [3]}
    gbm_reg_grid = model_selection.GridSearchCV(gbm_reg, gbm_reg_param_grid,cv=10, n_jobs=25, verbose=1,scoring='neg_mean_squared_error')
    gbm_reg_grid.fit(missing_age_X_train, missing_age_Y_train)
    print('Age feature Best GB Params:' + str(gbm_reg_grid.best_params_))
    print('Age feature Best GB Score:' + str(gbm_reg_grid.best_score_))
    print('GB Train Error for "Age" Feature Regressor:'+ str(gbm_reg_grid.score(missing_age_X_train, missing_age_Y_train)))
    missing_age_test['Age_GB'] = gbm_reg_grid.predict(missing_age_X_test)
    print(missing_age_test['Age_GB'][:4])

    lrf_reg = LinearRegression()
    lrf_reg_param_grid = {'fit_intercept': [True], 'normalize': [True]}
    lrf_reg_grid = model_selection.GridSearchCV(lrf_reg, lrf_reg_param_grid,cv=10, n_jobs=25, verbose=1,scoring='neg_mean_squared_error')
    lrf_reg_grid.fit(missing_age_X_train, missing_age_Y_train)
    print('Age feature Best LR Params:' + str(lrf_reg_grid.best_params_))
    print('Age feature Best LR Score:' + str(lrf_reg_grid.best_score_))
    print('LR Train Error for "Age" Feature Regressor' + str(lrf_reg_grid.score(missing_age_X_train, missing_age_Y_train)))

    missing_age_test['Age_LRF'] = lrf_reg_grid.predict(missing_age_X_test)
    print(missing_age_test['Age_LRF'][:4])

    print('shape1',missing_age_test['Age'].shape,missing_age_test[['Age_GB','Age_LRF']].mode(axis=1).shape)
    #missing_age_test['Age'] = missing_age_test[['Age_GB','Age_LRF']].mode(axis=1)
    missing_age_test['Age'] = np.mean([missing_age_test['Age_GB'],missing_age_test['Age_LRF']])
    print(missing_age_test['Age'][:4])
    drop_col_not_req(missing_age_test, ['Age_GB', 'Age_LRF'])

    return missing_age_test

In [4]:
# 筛选重要特征
def get_top_n_features(titanic_train_data_X, titanic_train_data_Y, top_n_features):
    # 随机森林
    rf_est = RandomForestClassifier(random_state=42)
    rf_param_grid = {'n_estimators': [500], 'min_samples_split': [2, 3], 'max_depth': [20]}
    rf_grid = model_selection.GridSearchCV(rf_est, rf_param_grid, n_jobs=25, cv=10, verbose=1)
    rf_grid.fit(titanic_train_data_X,titanic_train_data_Y)
    print('Top N Features Best RF Params:' + str(rf_grid.best_params_))
    print('Top N Features Best RF Score:' + str(rf_grid.best_score_))
    print('Top N Features RF Train Error:' + str(rf_grid.score(titanic_train_data_X, titanic_train_data_Y)))
    feature_imp_sorted_rf = pd.DataFrame({'feature': list(titanic_train_data_X), 
                            'importance': rf_grid.best_estimator_.feature_importances_}).sort_values('importance', ascending=False)
    features_top_n_rf = feature_imp_sorted_rf.head(top_n_features)['feature']
    print('Sample 25 Features from RF Classifier')
    print(str(features_top_n_rf[:25]))

    # AdaBoost
    ada_est = ensemble.AdaBoostClassifier(random_state=42)
    ada_param_grid = {'n_estimators': [500], 'learning_rate': [0.5, 0.6]}
    ada_grid = model_selection.GridSearchCV(ada_est, ada_param_grid, n_jobs=25, cv=10, verbose=1)
    ada_grid.fit(titanic_train_data_X, titanic_train_data_Y)
    print('Top N Features Best Ada Params:' + str(ada_grid.best_params_))
    print('Top N Features Best Ada Score:' + str(ada_grid.best_score_))
    print('Top N Features Ada Train Error:' + str(ada_grid.score(titanic_train_data_X, titanic_train_data_Y)))
    feature_imp_sorted_ada = pd.DataFrame({'feature': list(titanic_train_data_X),
                            'importance': ada_grid.best_estimator_.feature_importances_}).sort_values('importance', ascending=False)
    features_top_n_ada = feature_imp_sorted_ada.head(top_n_features)['feature']
    print('Sample 25 Feature from Ada Classifier:')
    print(str(features_top_n_ada[:25]))

    # ExtraTree
    et_est = ensemble.ExtraTreesClassifier(random_state=42)
    et_param_grid = {'n_estimators': [500], 'min_samples_split': [3, 4], 'max_depth': [15]}
    et_grid = model_selection.GridSearchCV(et_est, et_param_grid, n_jobs=25, cv=10, verbose=1)
    et_grid.fit(titanic_train_data_X, titanic_train_data_Y)
    print('Top N Features Best ET Params:' + str(et_grid.best_params_))
    print('Top N Features Best ET Score:' + str(et_grid.best_score_))
    print('Top N Features ET Train Error:' + str(et_grid.score(titanic_train_data_X, titanic_train_data_Y)))
    feature_imp_sorted_et = pd.DataFrame({'feature': list(titanic_train_data_X),
                            'importance': et_grid.best_estimator_.feature_importances_}).sort_values('importance', ascending=False)
    features_top_n_et = feature_imp_sorted_et.head(top_n_features)['feature']
    print('Sample 25 Features from ET Classifier:')
    print(str(features_top_n_et[:25]))

    # 融合以上三个模型
    features_top_n = pd.concat([features_top_n_rf, features_top_n_ada, features_top_n_et],ignore_index=True).drop_duplicates()

    return features_top_n

In [5]:
# 开始训练模型
train_data_org = pd.read_csv('train.csv')
test_data_org = pd.read_csv('test.csv')

# 将训练集和测试集合并
test_data_org['Survived'] = 0
combined_train_test = train_data_org.append(test_data_org)

D:\Users\WYG\Anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [6]:
# 特征工程

# 1. Embarked
# Embarkde中有两个NaN
if combined_train_test['Embarked'].isnull().sum() != 0:
    combined_train_test['Embarked'].fillna(combined_train_test['Embarked'].mode().iloc[0], inplace=True)

In [7]:
emb_dummies_df = pd.get_dummies(combined_train_test['Embarked'],
                                prefix=combined_train_test[['Embarked']].columns[0])
combined_train_test = pd.concat([combined_train_test, emb_dummies_df], axis=1)

In [8]:
# 2. Sex
sex_dummies_df = pd.get_dummies(combined_train_test['Sex'], prefix=combined_train_test[['Sex']].columns[0])
combined_train_test = pd.concat([combined_train_test, sex_dummies_df], axis=1)

In [9]:
# 3. Name
# Title
combined_train_test['Title'] = combined_train_test['Name'].str.extract('.+,(.+)', expand=False).str.extract('^(.+?)\.', expand=False).str.strip()
print(combined_train_test['Title'].groupby(by=combined_train_test['Title']).count().sort_values(ascending=False))

Title
Mr              757
Miss            260
Mrs             197
Master           61
Dr                8
Rev               8
Col               4
Ms                2
Mlle              2
Major             2
Don               1
Dona              1
the Countess      1
Jonkheer          1
Lady              1
Sir               1
Mme               1
Capt              1
Name: Title, dtype: int64


In [10]:
title_Dict = {}
title_Dict.update(dict.fromkeys(['Capt', 'Col', 'Major', 'Dr', 'Rev'], 'Officer'))
title_Dict.update(dict.fromkeys(['Jonkheer', 'Don', 'Sir', 'the Countess', 'Dona', 'Lady'], 'Royalty'))
title_Dict.update(dict.fromkeys(['Mme', 'Ms', 'Mrs'], 'Mrs'))
title_Dict.update(dict.fromkeys(['Mlle', 'Miss'], 'Miss'))
title_Dict.update(dict.fromkeys(['Mr'], 'Mr'))
title_Dict.update(dict.fromkeys(['Master'], 'Master'))

combined_train_test['Title'] = combined_train_test['Title'].map(title_Dict)
print(combined_train_test['Title'].groupby(by=combined_train_test['Title']).count().sort_values(ascending=False))

Title
Mr         757
Miss       262
Mrs        200
Master      61
Officer     23
Royalty      6
Name: Title, dtype: int64


In [11]:
title_dummies_df = pd.get_dummies(combined_train_test['Title'], prefix=combined_train_test[['Title']].columns[0])
combined_train_test = pd.concat([combined_train_test, title_dummies_df], axis=1)
print(combined_train_test)

Age        Cabin Embarked      Fare  \
0    22.0          NaN        S    7.2500   
1    38.0          C85        C   71.2833   
2    26.0          NaN        S    7.9250   
3    35.0         C123        S   53.1000   
4    35.0          NaN        S    8.0500   
5     NaN          NaN        Q    8.4583   
6    54.0          E46        S   51.8625   
7     2.0          NaN        S   21.0750   
8    27.0          NaN        S   11.1333   
9    14.0          NaN        C   30.0708   
10    4.0           G6        S   16.7000   
11   58.0         C103        S   26.5500   
12   20.0          NaN        S    8.0500   
13   39.0          NaN        S   31.2750   
14   14.0          NaN        S    7.8542   
15   55.0          NaN        S   16.0000   
16    2.0          NaN        Q   29.1250   
17    NaN          NaN        S   13.0000   
18   31.0          NaN        S   18.0000   
19    NaN          NaN        C    7.2250   
20   35.0          NaN        S   26.0000   
21   34.0       

In [12]:
# Name Length
combined_train_test['Name_Length'] = combined_train_test['Name'].str.len()
combined_train_test['Name_length_Category'] = combined_train_test['Name_Length'].map(name_length_category)
print(combined_train_test['Name_length_Category'].groupby(by=combined_train_test['Name_length_Category']).count().sort_values(ascending=False))

le_name = LabelEncoder()
le_name.fit(np.array(['Very_Short_Name', 'Short_Name', 'Medium_Name', 'Long_Name']))
combined_train_test['Name_length_Category'] = le_name.transform(combined_train_test['Name_length_Category'])

print(combined_train_test[['Name_length_Category','Survived']].corr())
name_length_dummies_df = pd.get_dummies(combined_train_test['Name_length_Category'],
                                        prefix=combined_train_test[['Name_length_Category']].columns[0])
combined_train_test = pd.concat([combined_train_test, name_length_dummies_df], axis=1)
print(combined_train_test)

0000   
396  24.0          NaN        Q    7.2500   
397  48.0          B41        C   79.2000   
398  22.0          NaN        S    7.7750   
399  31.0          NaN        Q    7.7333   
400  30.0           C7        S  164.8667   
401  38.0          NaN        S   21.0000   
402  22.0          NaN        C   59.4000   
403  17.0          NaN        S   47.1000   
404  43.0          D40        C   27.7208   
405  20.0          D38        C   13.8625   
406  23.0          NaN        S   10.5000   
407  50.0          C80        C  211.5000   
408   NaN          NaN        Q    7.7208   
409   3.0          NaN        S   13.7750   
410   NaN          NaN        Q    7.7500   
411  37.0          C78        Q   90.0000   
412  28.0          NaN        S    7.7750   
413   NaN          NaN        S    8.0500   
414  39.0         C105        C  108.9000   
415  38.5          NaN        S    7.2500   
416   NaN          NaN        S    8.0500   
417   NaN          NaN        C   22.3583   

 

In [13]:
# First Name
combined_train_test['First_Name'] = combined_train_test['Name'].str.extract('^(.+?),', expand=False).str.strip()
first_name_dummies_df = pd.get_dummies(combined_train_test['First_Name'],
                                        prefix=combined_train_test[['First_Name']].columns[0])
combined_train_test = pd.concat([combined_train_test, first_name_dummies_df], axis=1)
print(combined_train_test)

 ...          ...   
388                               Canavan, Mr. Patrick      0         1280   
389                        Palsson, Master. Paul Folke      1         1281   
390                         Payne, Mr. Vivian Ponsonby      0         1282   
391     Lines, Mrs. Ernest H (Elizabeth Lindsey James)      1         1283   
392                      Abbott, Master. Eugene Joseph      2         1284   
393                               Gilbert, Mr. William      0         1285   
394                           Kink-Heilmann, Mr. Anton      1         1286   
395     Smith, Mrs. Lucien Philip (Mary Eloise Hughes)      0         1287   
396                               Colbert, Mr. Patrick      0         1288   
397  Frolicher-Stehli, Mrs. Maxmillian (Margaretha ...      1         1289   
398                     Larsson-Rondberg, Mr. Edvard A      0         1290   
399                           Conlon, Mr. Thomas Henry      0         1291   
400                            Bonnell, Mis

In [14]:
# Last_Name
combined_train_test['Last_Name'] = combined_train_test['Name'].str.split('\.').str[1].str.strip()
combined_train_test['Last_Name'] = combined_train_test['Last_Name'].str.strip('\([^)]*\)')
combined_train_test['Last_Name'].fillna(combined_train_test['Name'].str.split('\.').str[1].str.strip())
print(combined_train_test['Last_Name'].groupby(by = combined_train_test['Last_Name']).count().sort_values(ascending = False)[:5])
last_name_dummies_df = pd.get_dummies(combined_train_test['Last_Name'],
                                        prefix=combined_train_test[['Last_Name']].columns[0])
combined_train_test = pd.concat([combined_train_test, last_name_dummies_df], axis=1)

print(combined_train_test)

edro      0         1295   
404                       Frauenthal, Mr. Isaac Gerald      0         1296   
405       Nourney, Mr. Alfred (Baron von Drachstedt")"      0         1297   
406                          Ware, Mr. William Jeffery      0         1298   
407                         Widener, Mr. George Dunton      1         1299   
408                    Riordan, Miss. Johanna Hannah""      0         1300   
409                          Peacock, Miss. Treasteall      1         1301   
410                             Naughton, Miss. Hannah      0         1302   
411    Minahan, Mrs. William Edward (Lillian E Thorpe)      0         1303   
412                     Henriksson, Miss. Jenny Lovisa      0         1304   
413                                 Spector, Mr. Woolf      0         1305   
414                       Oliva y Ocana, Dona. Fermina      0         1306   
415                       Saether, Mr. Simon Sivertsen      0         1307   
416                                W

In [15]:
# Original_Name
combined_train_test['Original_Name'] = combined_train_test['Name'].str.split('\((.*?)\)').str[1].str.strip(
    '\"').str.strip()
print(combined_train_test['Original_Name'].groupby(by = combined_train_test['Original_Name']).count().sort_values(ascending = False)[:5])
original_name_dummies_df = pd.get_dummies(combined_train_test['Original_Name'],
                                            prefix=combined_train_test[['Original_Name']].columns[0])
combined_train_test = pd.concat([combined_train_test, original_name_dummies_df], axis=1)

Original_Name
Mary                          2
Emma                          2
Emelia Maria Vandemoortele    1
Ellen Truelove Arman          1
Ellen Needs                   1
Name: Original_Name, dtype: int64


In [16]:
# 4. Fare
# 填充NaN
if combined_train_test['Fare'].isnull().sum() != 0:
    combined_train_test['Fare'] = combined_train_test[['Fare']].fillna(
        combined_train_test.groupby('Pclass').transform('mean'))

In [17]:
# 将多人船票的价格平均到每人
combined_train_test['Group_Ticket'] = combined_train_test['Fare'].groupby(
    by=combined_train_test['Ticket']).transform('count')
combined_train_test['Fare'] = combined_train_test['Fare'] / combined_train_test['Group_Ticket']
combined_train_test.drop(['Group_Ticket'], axis=1, inplace=True)

In [18]:
# 去除Fare为0的项
if sum(n == 0 for n in combined_train_test.Fare.values.flatten()) > 0:
    combined_train_test.loc[combined_train_test.Fare == 0, 'Fare'] = np.nan
    combined_train_test['Fare'] = combined_train_test[['Fare']].fillna(
        combined_train_test.groupby('Pclass').transform('mean'))
print(combined_train_test['Fare'].describe())

count    1309.000000
mean       15.017884
std        13.529548
min         3.170800
25%         7.666667
50%         8.300000
75%        15.050000
max       128.082300
Name: Fare, dtype: float64


In [19]:
# 建立Fare Category
combined_train_test['Fare_Category'] = combined_train_test['Fare'].map(fare_category)
le_fare = LabelEncoder()
le_fare.fit(np.array(['Very_Low_Fare', 'Low_Fare', 'Med_Fare', 'High_Fare', 'Very_High_Fare']))
combined_train_test['Fare_Category'] = le_fare.transform(combined_train_test['Fare_Category'])

In [20]:
fare_cat_dummies_df = pd.get_dummies(combined_train_test['Fare_Category'],
                                        prefix=combined_train_test[['Fare_Category']].columns[0])
combined_train_test = pd.concat([combined_train_test, fare_cat_dummies_df], axis=1)
print(combined_train_test['Fare_Category'].groupby(by = combined_train_test['Fare_Category']).count().sort_values(ascending = False))

Fare_Category
1    745
2    408
0    112
3     40
4      4
Name: Fare_Category, dtype: int64


In [21]:
# 5. Pclass
print(combined_train_test['Fare'].groupby(by = combined_train_test['Pclass']).mean())
Pclass_1_mean_fare = combined_train_test['Fare'].groupby(by=combined_train_test['Pclass']).mean().get([1]).values[0]
Pclass_2_mean_fare = combined_train_test['Fare'].groupby(by=combined_train_test['Pclass']).mean().get([2]).values[0]
Pclass_3_mean_fare = combined_train_test['Fare'].groupby(by=combined_train_test['Pclass']).mean().get([3]).values[0]
# 建立Pclass_Fare Category
combined_train_test['Pclass_Fare_Category'] = combined_train_test.apply(pclass_fare_category, args=(
Pclass_1_mean_fare, Pclass_2_mean_fare, Pclass_3_mean_fare), axis=1)
print(combined_train_test['Pclass_Fare_Category'].groupby(by = combined_train_test['Pclass_Fare_Category']).count().sort_values(ascending = False))
p_fare = LabelEncoder()
p_fare.fit(np.array(
    ['Pclass_1_Low_Fare', 'Pclass_1_High_Fare', 'Pclass_2_Low_Fare', 'Pclass_2_High_Fare', 'Pclass_3_Low_Fare',
        'Pclass_3_High_Fare']))
combined_train_test['Pclass_Fare_Category'] = p_fare.transform(combined_train_test['Pclass_Fare_Category'])


Pclass
1    34.661682
2    11.663652
3     7.379203
Name: Fare, dtype: float64
Pclass_Fare_Category
Pclass_3_High_Fare    432
Pclass_3_Low_Fare     277
Pclass_1_Low_Fare     209
Pclass_2_High_Fare    155
Pclass_2_Low_Fare     122
Pclass_1_High_Fare    114
Name: Pclass_Fare_Category, dtype: int64


In [22]:
# 6. Parch and SibSp
combined_train_test['Family_Size'] = combined_train_test['Parch'] + combined_train_test['SibSp'] + 1
combined_train_test['Family_Size_Category'] = combined_train_test['Family_Size'].map(family_size_category)

le_family = LabelEncoder()
le_family.fit(np.array(['Single', 'Small_Family', 'Large_Family']))
combined_train_test['Family_Size_Category'] = le_family.transform(combined_train_test['Family_Size_Category'])

fam_size_cat_dummies_df = pd.get_dummies(combined_train_test['Family_Size_Category'],
                                            prefix=combined_train_test[['Family_Size_Category']].columns[0])
combined_train_test = pd.concat([combined_train_test, fam_size_cat_dummies_df], axis=1)
print(combined_train_test['Family_Size_Category'].groupby(by = combined_train_test['Family_Size_Category']).count().sort_values(ascending = False))

Family_Size_Category
1    790
2    394
0    125
Name: Family_Size_Category, dtype: int64


In [23]:
# 7. Age
# 填充Age中的NaN值
combined_train_test['Age_Null'] = combined_train_test['Age'].apply(lambda x: 1 if pd.notnull(x) else 0)

missing_age_df = pd.DataFrame(
    combined_train_test[['Age', 'Parch', 'Sex', 'SibSp', 'Family_Size', 'Family_Size_Category',
                            'Title', 'Fare', 'Fare_Category', 'Pclass', 'Embarked']])
missing_age_df = pd.get_dummies(missing_age_df,columns=['Title', 'Family_Size_Category', 'Fare_Category', 'Sex', 'Pclass','Embarked'])
missing_age_train = missing_age_df[missing_age_df['Age'].notnull()]
missing_age_test = missing_age_df[missing_age_df['Age'].isnull()]
combined_train_test.loc[(combined_train_test.Age.isnull()), 'Age'] = fill_missing_age(missing_age_train, missing_age_test)
print(combined_train_test.describe())

Fitting 10 folds for each of 1 candidates, totalling 10 fits
[Parallel(n_jobs=25)]: Using backend LokyBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done   5 out of  10 | elapsed:    6.0s remaining:    6.0s
[Parallel(n_jobs=25)]: Done  10 out of  10 | elapsed:    6.1s finished
D:\Users\WYG\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
Age feature Best GB Params:{'learning_rate': 0.01, 'max_depth': 3, 'max_features': 3, 'n_estimators': 2000}
Age feature Best GB Score:-112.55495379252793
GB Train Error for "Age" Feature Regressor:-91.33225559467515
5     33.501856
17    33.285687
19    33.233397
26    26.619434
Name: Age_GB, dtype: float64
Fitting 10 folds for each of 1 candidates, totalling 10 fits
D:\Users\WYG\Anaconda3\lib\site-packages

In [24]:
#检查是否有异常值
if sum(n<0 for n in combined_train_test.Age.values.flatten()) > 0:
    combined_train_test.loc[combined_train_test.Age < 0,'Age'] = np.nan
    combined_train_test['Age'] = combined_train_test[['Age']].fillna(combined_train_test.groupby('Title').transform('mean'))
print(combined_train_test['Age'].groupby(by=combined_train_test['Title']).mean().sort_values(ascending=True))

# 建立Age_Category
combined_train_test['Age_Category'] = combined_train_test['Age'].map(age_group_category)
le_age = LabelEncoder()
le_age.fit(np.array(['Baby','Toddler','Child','Teenager','Adult','Middle_Aged','Senior_Citizen','Old']))
combined_train_test['Age_Category'] = le_age.transform(combined_train_test['Age_Category'])
age_cat_dummies_df = pd.get_dummies(combined_train_test['Age_Category'],
                                    prefix=combined_train_test[['Age_Category']].columns[0])
combined_train_test = pd.concat([combined_train_test,age_cat_dummies_df],axis=1)

Title
Master      8.643103
Miss       23.281099
Mr         31.631154
Mrs        35.846362
Royalty    41.166667
Officer    45.547007
Name: Age, dtype: float64


In [25]:
#8. Ticket
combined_train_test['Ticket_Letter'] = combined_train_test['Ticket'].str.split().str[0]
combined_train_test['Ticket_Letter'] = combined_train_test['Ticket_Letter'].apply(lambda x:np.nan if x.isnumeric() else x)
combined_train_test['Ticket_Number'] = combined_train_test['Ticket'].apply(lambda x: pd.to_numeric(x,errors='coerce'))
combined_train_test['Ticket_Number'].fillna(0,inplace=True)
combined_train_test = pd.get_dummies(combined_train_test,columns=['Ticket','Ticket_Letter'])
print(combined_train_test.shape)

(1309, 3253)


In [26]:
#9. Cabin
combined_train_test['Cabin_Letter'] = combined_train_test['Cabin'].apply(lambda x:str(x)[0] if pd.notnull(x) else x)
combined_train_test = pd.get_dummies(combined_train_test,columns=['Cabin','Cabin_Letter'])
print(combined_train_test.shape)

(1309, 3446)


In [27]:
#10. 将Age和Fare正则化
scale_age_fare = preprocessing.StandardScaler().fit(combined_train_test[['Age','Fare']])
combined_train_test[['Age','Fare']] = scale_age_fare.transform(combined_train_test[['Age','Fare']])

In [28]:
#11. 弃掉无用列
combined_train_test.drop(['Name', 'PassengerId', 'Embarked', 'Sex', 'Title', 'Fare_Category',
                    'Family_Size_Category', 'Age_Category', 'First_Name', 'Last_Name',
                    'Original_Name', 'Name_length_Category'],axis=1,inplace=True)
print(combined_train_test.describe())

Age          Fare        Parch       Pclass        SibSp  \
count  1.309000e+03  1.309000e+03  1309.000000  1309.000000  1309.000000   
mean  -3.906401e-16 -7.667239e-17     0.385027     2.294882     0.498854   
std    1.000382e+00  1.000382e+00     0.865560     0.837836     1.041658   
min   -2.302295e+00 -8.759798e-01     0.000000     1.000000     0.000000   
25%   -6.072652e-01 -5.435530e-01     0.000000     2.000000     0.000000   
50%   -1.861265e-02 -4.967240e-01     0.000000     3.000000     0.000000   
75%    4.021431e-01  2.374692e-03     0.000000     3.000000     1.000000   
max    3.896249e+00  8.360045e+00     9.000000     3.000000     8.000000   

          Survived   Embarked_C   Embarked_Q   Embarked_S   Sex_female  ...  \
count  1309.000000  1309.000000  1309.000000  1309.000000  1309.000000  ...   
mean      0.261268     0.206264     0.093965     0.699771     0.355997  ...   
std       0.439494     0.404777     0.291891     0.458533     0.478997  ...   
min       0.000

In [29]:
#12. 整理数据
train_data = combined_train_test[:891]
test_data = combined_train_test[891:]

titanic_train_data_X = train_data.drop(['Survived'],axis=1)
titanic_train_data_Y = train_data['Survived']

titanic_test_data_X = test_data.drop(['Survived'],axis=1)

In [30]:
#13. 利用特征值重要性排名来去除无用列
feature_to_pick = 250
feature_top_n = get_top_n_features(titanic_train_data_X,titanic_train_data_Y,feature_to_pick)

Fitting 10 folds for each of 2 candidates, totalling 20 fits
[Parallel(n_jobs=25)]: Using backend LokyBackend with 25 concurrent workers.
[Parallel(n_jobs=25)]: Done  13 out of  20 | elapsed:   12.5s remaining:    6.7s
[Parallel(n_jobs=25)]: Done  20 out of  20 | elapsed:   15.2s finished
Top N Features Best RF Params:{'max_depth': 20, 'min_samples_split': 2, 'n_estimators': 500}
Top N Features Best RF Score:0.835016835016835
Top N Features RF Train Error:0.936026936026936
Sample 25 Features from RF Classifier
9                     Sex_male
8                   Sex_female
12                    Title_Mr
16                 Name_Length
1                         Fare
2245      Pclass_Fare_Category
11                  Title_Miss
13                   Title_Mrs
3                       Pclass
0                          Age
2259             Ticket_Number
2246               Family_Size
2241           Fare_Category_1
17      Name_length_Category_0
2249    Family_Size_Category_2
4                  

In [31]:
print('Total Feature:'+str(combined_train_test.shape))
print('Picked Feature'+str(feature_top_n.shape))

titanic_train_data_X = titanic_train_data_X[feature_top_n]
del titanic_train_data_X['Ticket_Number']
titanic_test_data_X = titanic_test_data_X[feature_top_n]
del titanic_test_data_X['Ticket_Number']

Total Feature:(1309, 3434)
Picked Feature(438,)


In [32]:
#14.建立模型
rf_est = ensemble.RandomForestClassifier(n_estimators = 750, criterion = 'gini', max_features = 'sqrt',
                                            max_depth = 3, min_samples_split = 4, min_samples_leaf = 2,
                                            n_jobs = 50, random_state = 42, verbose = 1)
gbm_est = ensemble.GradientBoostingClassifier(n_estimators=900, learning_rate=0.0008, loss='exponential',
                                                min_samples_split=3, min_samples_leaf=2, max_features='sqrt',
                                                max_depth=3, random_state=42, verbose=1)
et_est = ensemble.ExtraTreesClassifier(n_estimators=750, max_features='sqrt', max_depth=35, n_jobs=50,
                                        criterion='entropy', random_state=42, verbose=1)

voting_est = ensemble.VotingClassifier(estimators = [('rf', rf_est),('gbm', gbm_est),('et', et_est)],
                                    voting = 'soft', weights = [4,0,4],
                                    n_jobs = 50)
voting_est.fit(titanic_train_data_X,titanic_train_data_Y)
print('VotingClassifier Score:' + str(voting_est.score(titanic_train_data_X,titanic_train_data_Y)))
print('VotingClassifier Estimators:' + str(voting_est.estimators_))

[Parallel(n_jobs=50)]: Using backend ThreadingBackend with 50 concurrent workers.
[Parallel(n_jobs=50)]: Done 100 tasks      | elapsed:    0.0s
[Parallel(n_jobs=50)]: Done 350 tasks      | elapsed:    0.0s
[Parallel(n_jobs=50)]: Done 750 out of 750 | elapsed:    0.1s finished
[Parallel(n_jobs=50)]: Using backend ThreadingBackend with 50 concurrent workers.
[Parallel(n_jobs=50)]: Done 100 tasks      | elapsed:    0.0s
[Parallel(n_jobs=50)]: Done 350 tasks      | elapsed:    0.0s
[Parallel(n_jobs=50)]: Done 750 out of 750 | elapsed:    0.1s finished
VotingClassifier Score:0.9966329966329966
VotingClassifier Estimators:[RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=3, max_features='sqrt', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=4,
                       min_weight_fraction_leaf=0.0, n_estimators=750,
           

In [33]:
#预测
titanic_test_data_X['Survived'] = voting_est.predict(titanic_test_data_X)

submission = pd.DataFrame({'PassengerId':test_data_org.loc[:,'PassengerId'],
                            'Survived':titanic_test_data_X.loc[:,'Survived']})

submission.to_csv('result4.csv',index=False,sep=',')

[Parallel(n_jobs=50)]: Using backend ThreadingBackend with 50 concurrent workers.
[Parallel(n_jobs=50)]: Done 100 tasks      | elapsed:    0.0s
[Parallel(n_jobs=50)]: Done 350 tasks      | elapsed:    0.0s
[Parallel(n_jobs=50)]: Done 750 out of 750 | elapsed:    0.0s finished
[Parallel(n_jobs=50)]: Using backend ThreadingBackend with 50 concurrent workers.
[Parallel(n_jobs=50)]: Done 100 tasks      | elapsed:    0.0s
[Parallel(n_jobs=50)]: Done 350 tasks      | elapsed:    0.0s
[Parallel(n_jobs=50)]: Done 750 out of 750 | elapsed:    0.0s finished
